### CODE FROM
https://github.com/smortezavi/Randomized_SVD_GPU/blob/master/pytorch_randomized_svd.ipynb

In [1]:
import numpy as np
import timeit
import pandas as pd
from sklearn import decomposition
#import fbpca
import torch
from scipy import linalg

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [13]:
def simple_randomized_svd(M, k=10):
    m, n = M.shape
    transpose = False
    if m < n:
        transpose = True
        M = M.T
    rand_matrix = np.random.normal(size=(M.shape[1], k))  # short side by k
    Q, _ = np.linalg.qr(M @ rand_matrix, mode='reduced')  # long side by k
    smaller_matrix = Q.T @ M                              # k by short side
    U_hat, s, V = np.linalg.svd(smaller_matrix, full_matrices=False)
    U = Q @ U_hat
    
    if transpose:
        return V.T, s.T, U.T
    else:
        return U, s, V

In [14]:
def simple_randomized_torch_svd(M, k=10):
    B = torch.tensor(M).cuda(0)
    m, n = B.size()
    transpose = False
    if m < n:
        transpose = True
        B = B.transpose(0, 1).cuda(0)
        m, n = B.size()
    rand_matrix = torch.rand((n,k), dtype=torch.double).cuda(0)  # short side by k
    Q, _ = torch.qr(B @ rand_matrix)                              # long side by k
    Q.cuda(0)
    smaller_matrix = (Q.transpose(0, 1) @ B).cuda(0)             # k by short side
    U_hat, s, V = torch.svd(smaller_matrix,False)
    U_hat.cuda(0)
    U = (Q @ U_hat)
    
    if transpose:
        return V.transpose(0, 1), s, U.transpose(0, 1)
    else:
        return U, s, V

In [3]:
# computes an orthonormal matrix whose range approximates the range of A
# power_iteration_normalizer can be safe_sparse_dot (fast but unstable), LU (imbetween), or QR (slow but most accurate)
def randomized_range_finder(A, size, n_iter=5):
    Q = np.random.normal(size=(A.shape[1], size))
    
    for i in range(n_iter):
        Q, _ = linalg.lu(A @ Q, permute_l=True)
        Q, _ = linalg.lu(A.T @ Q, permute_l=True)
        
    Q, _ = linalg.qr(A @ Q, mode='economic')
    return Q

In [3]:
def randomized_svd(M, n_components, n_oversamples=10, n_iter=4):
    n_random = n_components + n_oversamples
    
    Q = torch.tensor(randomized_range_finder(M, n_random, n_iter)).to(device)
    # project M to the (k + p) dimensional space using the basis vectors
    M = torch.tensor(M).to(device)
    B = Q.transpose(0, 1) @ M
    # compute the SVD on the thin matrix: (k + p) wide
    Uhat, s, V = torch.linalg.svd(B, full_matrices=False)
    Uhat, s, V = Uhat.to(device), s.to(device), V.to(device)
    Uhat = torch.tensor(Uhat).cuda(0)
    del B
    U = Q @ Uhat
    
    return U[:, :n_components], s[:n_components], V[:n_components, :]

In [17]:
def randomized_svd_original(M, n_components, n_oversamples=10, n_iter=4):
    
    n_random = n_components + n_oversamples
    
    Q = randomized_range_finder(M, n_random, n_iter)
    # project M to the (k + p) dimensional space using the basis vectors
    B = Q.T @ M
    # compute the SVD on the thin matrix: (k + p) wide
    Uhat, s, V = torch.linalg.svd(B, full_matrices=False)
    del B
    U = Q @ Uhat
    
    return U[:, :n_components], s[:n_components], V[:n_components, :]

In [4]:
A = np.random.uniform(-40,40,[10,100]) 
randomized_svd(A, 5)

NameError: name 'randomized_range_finder' is not defined

In [22]:
A = np.random.uniform(-40,40,[10,100]) 
simple_randomized_svd(A)

(array([[ 0.19328677, -0.21847277,  0.28376989, -0.57122888, -0.10004364,
          0.22829094, -0.08339047, -0.48915565, -0.28642758, -0.34305776],
        [-0.34004255, -0.2786585 ,  0.12955247,  0.26386793,  0.29396367,
          0.39145186,  0.45585845, -0.08654062, -0.46238558,  0.22707829],
        [-0.36137593,  0.25006138,  0.49025373,  0.06608449, -0.03950225,
         -0.39132525, -0.44752797, -0.01299266, -0.42399456,  0.16505606],
        [-0.19918044,  0.04024894, -0.54069772, -0.04130097, -0.25896515,
          0.46063734, -0.48018639,  0.19447736, -0.34191997, -0.00938017],
        [-0.17442737,  0.42248297, -0.39656404, -0.51842396,  0.31757506,
         -0.29363454,  0.33452528,  0.01422339, -0.25654489, -0.00754198],
        [-0.18027295,  0.46295192,  0.19851508,  0.24946512, -0.12990787,
          0.21348223,  0.22358329,  0.14480559, -0.01043853, -0.71973486],
        [-0.4298087 ,  0.22694889,  0.27148236, -0.39028725,  0.00788997,
          0.42429136, -0.0306724

In [23]:
A = np.random.uniform(-40,40,[10,100])
simple_randomized_torch_svd(A)

/tmp/ipykernel_479856/3982458943.py:10: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352645774/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2491.)
  Q, _ = torch.qr(B @ rand_matrix)                              # long side by k


(tensor([[ 1.4607e-01, -2.3617e-01,  2.9033e-01,  5.4967e-01, -5.0578e-02,
           2.6425e-01,  4.0639e-01,  4.5727e-01, -2.9379e-01, -5.9240e-02],
         [ 5.9151e-01, -6.2960e-02, -1.1366e-01,  1.8271e-01,  7.0003e-01,
          -2.9231e-01, -1.5539e-01,  1.0576e-02, -5.9213e-03, -8.8455e-03],
         [-4.9256e-01, -2.4273e-02, -2.9125e-02,  3.2931e-01,  2.4946e-01,
           1.8681e-02, -3.9019e-01,  1.0505e-01, -1.6694e-01,  6.2751e-01],
         [-1.4606e-01, -8.6853e-02,  7.0085e-01, -1.6375e-01,  2.3062e-01,
           7.3319e-02, -3.1125e-01,  2.6019e-01,  4.2780e-01, -2.1669e-01],
         [-1.8096e-01, -5.5440e-01, -1.5796e-01,  3.5604e-01,  2.8835e-02,
          -3.2172e-02,  2.1525e-01, -3.5910e-01,  5.7495e-01, -2.1254e-02],
         [-1.3625e-01,  3.4298e-01, -2.7870e-01, -2.7488e-02,  4.4540e-01,
           7.1174e-01,  1.8246e-01,  6.6158e-04,  1.7305e-01, -1.3106e-01],
         [ 1.3809e-01, -5.2281e-01,  2.1117e-01, -4.5750e-01,  1.6083e-01,
           3.5544e-